# Введение

Работа по освоению линейной регрессии. Для этого считаю необходимым написать базовый алгоритм самостоятельно, а потом сравнить его с готовым из пакета sklearn. Для работы буду использовать модель линейной регрессии с L2 регуляризацией и стохастическим градиентным спуском в виде оптимизатора. 


### Импорт необходимых пакетов

In [1]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from numpy import linalg

### Инициализация констант и генерация данных для анализа

In [2]:
n_features = 2
RANDOM_SEED = 43
n_objects = 300
eps = 1e-3

w_true = np.random.normal(size=(n_features, ))
X = np.random.uniform(-5, 5, (n_objects, n_features))
X[:, -1] = X[:, -2] + np.random.uniform(-eps, eps, X[:, -2].shape)
Y = X.dot(w_true) + np.random.normal(0, 1, (n_objects))

#### Разделим данные на тестовые и тренировочные выборки 

In [3]:
train_X,test_X,train_Y,test_Y = train_test_split(X,
                                                 Y,
                                                 test_size = 0.33,
                                                 random_state = RANDOM_SEED)

###  Вывод формул

Как я и говорил выше, буду использовать L2 регуляризацию.  
Выпишем изначальную формулу функции потерь:  
$$
    Q(X) = ||Y-Xw||_2^2 + \lambda^2||w||_2^2
$$
Преобразуем данную формулу:  
$$
    Q(X) = (Y - Xw)^T(Y-Xw) + \lambda^2(w)^T(w)  \rightarrow min  
$$

$$
    Q(X) = Y^TY - (Xw)^TY - Y^T(Xw) + (Xw)^T(Xw) + \lambda^2(w)^T(w) \rightarrow min  
$$
Найдем производную:
$$
    \nabla_w Q(X) = 0 - XY^T - XY^T + 2XX^Tw^T + 2\lambda^2w^T
$$
Прировняем к нулю и выразим вектор весов:  

$$  
    - 2XY^T  + 2XX^Tw^T + 2\lambda^2w^T = 0  
$$  
Упростив уравнение продолжим выражать вектор: 

$$  
    -XY^T  + XX^Tw^T + \lambda^2w^T = 0  
$$  
  
$$  
     XX^Tw^T + \lambda^2w^T = XY^T  
$$  
  
$$  
     (XX^T + \lambda^2)w^T = XY^T  
$$  
  
$$  
     w^T = (XX^T + \lambda^2)^{-1}XY^T  
$$  
  
$$  
     w = (X^TX + \lambda^2)^{-1}X^TY  
$$  

Теперь выпишу формулу градиентного спуска:  
$$  
    w^{(t+1)} = w^{(t)} - \eta  \nabla Q(w^{(t)})  
$$  
Где Q(w): 
$$  
    Q(w) =  2((X^TX + \lambda^2)w- X^TY)  
$$  

In [4]:
class LinearRegressionSGD(BaseEstimator, RegressorMixin):
    """LinearRegression with L2 regularization and SGD optimizer
    """
    def __init__(self,alpha: float= 0.0001,
                 max_steps = 1000,
                 lr = 1e-2, 
                 batch_size = 50,
                 random_seed = 43,
                ):
        self.alpha = alpha
        self.max_steps = max_steps
        self.lr =  lr
        self.batch_size = batch_size
        self.random_seed = random_seed
        
    def score(self,design_matrix, target):
        return (linalg.norm(target-design_matrix.dot(self.w), ord = 2))**2
        
    def fit(self,design_matrix, target):
        w = np.random.randn(design_matrix.shape[1])
        step = 0
        while (self.max_steps >= step):
            slice_num = np.random.randint(design_matrix.shape[0],size = self.batch_size)
            w -= 2 * self.lr * (np.dot(np.dot(design_matrix[slice_num].T,design_matrix[slice_num])+(self.alpha**2)*np.eye(design_matrix[slice_num].shape[1]),w) - np.dot(design_matrix[slice_num].T,target[slice_num])) / target[slice_num].shape[0]
            step += 1
        self.w = w
        return self
    
    def predict(self,design_matrix):
        return design_matrix@self.w

In [5]:
pipe = make_pipeline(StandardScaler(),LinearRegressionSGD(max_steps = 1000))

In [6]:
pipe.fit(train_X,train_Y)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearregressionsgd',
                 LinearRegressionSGD(alpha=0.0001, batch_size=50, lr=0.01,
                                     max_steps=1000, random_seed=43))],
         verbose=False)

In [7]:
predict = pipe.predict(test_X)

In [8]:
loss_handmade = sum((test_Y - predict)**2)

### Реализуем тот же алгорим, но только при помощи пакетных методов sklearn

In [9]:
package_pipe = make_pipeline(StandardScaler(),SGDRegressor(max_iter = 1000))

In [10]:
package_pipe.fit(train_X,train_Y)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('sgdregressor',
                 SGDRegressor(alpha=0.0001, average=False, early_stopping=False,
                              epsilon=0.1, eta0=0.01, fit_intercept=True,
                              l1_ratio=0.15, learning_rate='invscaling',
                              loss='squared_loss', max_iter=1000,
                              n_iter_no_change=5, penalty='l2', power_t=0.25,
                              random_state=None, shuffle=True, tol=0.001,
                              validation_fraction=0.1, verbose=0,
                              warm_start=False))],
         verbose=False)

In [11]:
predict_package = package_pipe.predict(test_X)

In [12]:
loss_package = sum((test_Y - predict_package)**2)

## Итоги

Посмотрим разницу между предсказаниями сделанными самописным алгритмом и классом SGDRegressor из sklearn

In [13]:
print('Ошибка полученная на предсказаниях проведенныйх ручным алгоритмом {:.3f} и алгоритмом sklearn {:.3f}'.format(loss_handmade, loss_package))
print("Разница между результатами составила: {:.3}".format(abs(loss_handmade - loss_package)))

Ошибка полученная на предсказаниях проведенныйх ручным алгоритмом 87.693 и алгоритмом sklearn 86.339
Разница между результатами составила: 1.35
